# NLP A3

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

### Data Access and Setup

Google Colab Mounting

In [1]:
"""from google.colab import drive  # importing drive module
drive.mount('/content/drive') # mounts drive to my google drive"""

"from google.colab import drive  # importing drive module\ndrive.mount('/content/drive') # mounts drive to my google drive"

In [2]:
"""%cd /content/drive/MyDrive/Colab Notebooks/NLP_A3
# change directory to the folder I created for project to work out of...
# didn't realise this comment was causing bugs when on the line above for a while :)
!pwd  # print working directory to double check
!ls # seeing files within folder"""

"%cd /content/drive/MyDrive/Colab Notebooks/NLP_A3\n# change directory to the folder I created for project to work out of...\n# didn't realise this comment was causing bugs when on the line above for a while :)\n!pwd  # print working directory to double check\n!ls # seeing files within folder"

### Library Installion

datasets: HF library for HF datasets

evaluate: HF library for evaluating HF models

transformers[sentenpiece]: HF transformers library with sentenpiece support (models that use sentenpiece) instead of wordpiece tokenization

accelerate: HF enabled library to simplify training across models, and provide improved efficiency methods

In [3]:
"""!pip install datasets evaluate 'transformers[sentencepiece]'
!pip install accelerate"""

"!pip install datasets evaluate 'transformers[sentencepiece]'\n!pip install accelerate"

# Loading Dataset - Master Coaching Hurstville

Importing custom data

In [4]:
import pandas as pd
df = pd.read_csv('data/ver2_1.csv') # converts CSV file into pandas dataframe to be used
print(f"Length DF: {len(df)}")  # showing length to double check against ID to easily make sure I got all/bugs
df.head(5)  # shows top 5 to quickly see data looks correct

Length DF: 61


,id,title,context,question,answers
0,0,Subjects offered,"""At our afterschool tuition center, we offer t...",What subjects do you offer tutoring in,"English, Mathematics, Chemistry, Biology, and ..."
1,1,Subjects offered,"""Our dedicated teachers specialize in providin...",Which core subjects are available at your centre,"English, Maths, and Science"
2,2,Subjects offered,"""Students at our academy can enroll for suppor...",Which academic subjects do you provide support...,"English Literature, Advanced Mathematics, Chem..."
3,3,Subjects offered,"""Our afterschool programs include Mathematics ...",Which programs and subjcets are included in yo...,"Mathematics tutoring, English writing workshop..."
4,4,Subjects offered,"""We provide specialist tutoring services in Ch...",What tutoring services do you offer in terms o...,"Chemistry, Biology, Physics, English Language,..."


In [5]:
df = df.dropna() # drop NaN rows (indexes are pre-filled so there's a few)
print(f"Length DF: {len(df)}")

Length DF: 49


# Editing dataset to be format expected by model

In [6]:
from datasets import Dataset

my_data = []  # initialise empty list to build in

for idx, row in df.iterrows():  # loop each item in the df (dataset)
    context = row['context']  # assigning context
    answer_text = row['answers']  # assigning answer

    try:
        answer_start = context.index(answer_text) # find character position of answer in context
    except ValueError:  # generic error identification - shouldn't happen in mine with manual dataset
        print(idx)
        print(f"❗ Warning: Could not find answer in context at row {idx}")
        continue  # Skip problematic row

    sample = {                    # creating sample (data point) per row
        "id": str(row['id']),
        "title": row['title'],
        "context": context,
        "question": row['question'],
        "answers": {
            "text": [answer_text],
            "answer_start": [answer_start]
        }
    }
    my_data.append(sample)  # add sample to list

dataset = Dataset.from_list(my_data)  # create hf dataset from list
print(dataset)  # quickly check dataset characteristics
dataset[0]  # Show first example to double check


c:\Users\petea\workspace\github.com\BL1TZau\NLP-41043-A3\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 49
})


{'id': '0',
 'title': 'Subjects offered',
 'context': '"At our afterschool tuition center, we offer tutoring in English, Mathematics, Chemistry, Biology, and Physics to help students excel in their academic journey."',
 'question': 'What subjects do you offer tutoring in',
 'answers': {'answer_start': [57],
  'text': ['English, Mathematics, Chemistry, Biology, and Physics']}}

### Creating train and test splits of the data

In [7]:
dataset = Dataset.from_list(my_data)  # create hf dataset
dataset = dataset.train_test_split(test_size=0.2, seed=42)  # create 80 train 20 split on dataset

MCH_train_dataset = dataset["train"]  # create training dataset
MCH_validation_dataset = dataset["test"]  # create testing dataset

# quick double check sizes of dataset
print(f"Training samples: {len(MCH_train_dataset)}")
print(f"Validation samples: {len(MCH_validation_dataset)}")
print(dataset)  # quick check dataset

Training samples: 39
Validation samples: 10
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 39
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10
    })
})


Quickly checking training samples to make sure it looks correct

In [8]:
for i in range(0, 2):
  print(f"--- Training Example {i} ---")
  print("ID: ", dataset["train"][i]["id"])
  print("title: ", dataset["train"][i]["title"])
  print("Context: ", dataset["train"][i]["context"])
  print("Question: ", dataset["train"][i]["question"])
  print("Answer: ", dataset["train"][i]["answers"])  # answer_start field contains the starting character index of each answer in the context
  print("Answer Text: ", dataset["train"][i]["answers"]['text'])  # answer_start field contains the starting character index of each answer in the context)
  print("")

--- Training Example 0 ---
ID:  18
title:  class type
Context:  Our entry into Year 7 English and Maths programs are designed to ensure a smooth and comfortable transition for the students. It is for the students who are finishing their Year 6 primary school and stepping into the Year 7 high school. Our Year 7 English Tutor and Year 7 Maths tutor, the major subjects which are usually of high concern for the students, conduct sessions for repeated revisions to master all the primary skills that are of high importance while stepping ahead in the high school. We introduce the Year 7 topics to the students slowly and steadily so that the students do not lag behind nor become shaky about it. As the students get used to the concepts and skills, the Year 7 English Tutor and Year 7 Maths tutor start nurturing and working on it further. Our tutors help the Year 6 students get accustomed to the techniques and methods used in the high school and help them learn to work on it
Question:  Do you off

Double checking test set too

In [9]:
for i in range(0, 2):
  print(f"--- Test Example {i} ---")
  print("ID: ", dataset["test"][i]["id"])
  print("title: ", dataset["test"][i]["title"])
  print("Context: ", dataset["test"][i]["context"])
  print("Question: ", dataset["test"][i]["question"])
  print("Answer: ", dataset["test"][i]["answers"])  # answer_start field contains the starting character index of each answer in the context
  print("Answer Text: ", dataset["test"][i]["answers"]['text'])  # answer_start field contains the starting character index of each answer in the context)
  print("")

--- Test Example 0 ---
ID:  24
title:  class type
Context:  Master Coaching is offering an exclusive Holiday Enhancement Course – designed for students who wish to hone their knowledge, and stay ahead of their classmates. The course is open to students from year 2 to year 12. It is the best way to enjoy your holidays, make the most of the festivities, as well as, upgrade and strengthen your school’s assignments and syllabus. Master Coaching’s Holiday Enhancement Course includes English and Math tutoring, along with courses to help in OC preparation and basic curriculum. Achieving academic success is the number one priority for both parents and students. Getting ahead in class and acquiring the highest ranking, is what every student aims for. Having said this, you can accomplish such a goal, if you get trained and assisted by the best. We at Master Coaching understand the value of student goals and their aspirations – and strive to guide our pupils towards the road of academic success. 

# Loading pre-trained model and AutoTokenizer

In [10]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
# AutoTokenizer: Loads correct tokenizer for any model checkpoint
""" AutoModelForQuestionAnswering: Loads the model architecture designed to output answer spans from context. """

model_checkpoint = "deepset/roberta-base-squad2"  # model selection

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) # loads tokenizer of specific model selected
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint) # loads QA model weights
tokenizer.is_fast # checks if model supports HF accelerator: rust backend implementation to vastly increase efficiency

True

Checking tokens visually

In [11]:
context = dataset["train"][0]["context"]    # use first sample as example
question = dataset["train"][0]["question"]
print(f"-- Question -- \n{question}")
print(f"-- Context --\n{context}\n")

print(f"-- Tokenized --")
tokens = tokenizer(question, context) # tokenize
print(tokens) # print token ids
print(f"-- Decoded --")
print(tokenizer.convert_ids_to_tokens(tokens['input_ids'])) # print actual tokens


-- Question -- 
Do you offer year 7 classes?
-- Context --
Our entry into Year 7 English and Maths programs are designed to ensure a smooth and comfortable transition for the students. It is for the students who are finishing their Year 6 primary school and stepping into the Year 7 high school. Our Year 7 English Tutor and Year 7 Maths tutor, the major subjects which are usually of high concern for the students, conduct sessions for repeated revisions to master all the primary skills that are of high importance while stepping ahead in the high school. We introduce the Year 7 topics to the students slowly and steadily so that the students do not lag behind nor become shaky about it. As the students get used to the concepts and skills, the Year 7 English Tutor and Year 7 Maths tutor start nurturing and working on it further. Our tutors help the Year 6 students get accustomed to the techniques and methods used in the high school and help them learn to work on it

-- Tokenized --
{'input_i

# Sliding window tokenization for longer context

In [12]:
inputs = tokenizer( # tokenizing
    question, # getting question
    context,  # getting answer
    max_length=100, # set token maximum length
    truncation="only_second",   # keeping Q intact - truncate the second input(context) if question length > max_length
    stride=50,  # allow model to see answers across splits - 50 tokens overlap between splits (are seen in both)
    return_overflowing_tokens=True, # Required to make stride work --returns chunks of tokens. Otherwise returns one max_length token
)
z=0
for ids in inputs["input_ids"]:
    z+=1
    print(f"Chunk: {z}, Len: {len(ids)}")
    print(tokenizer.decode(ids))

Chunk: 1, Len: 100
<s>Do you offer year 7 classes?</s></s>Our entry into Year 7 English and Maths programs are designed to ensure a smooth and comfortable transition for the students. It is for the students who are finishing their Year 6 primary school and stepping into the Year 7 high school. Our Year 7 English Tutor and Year 7 Maths tutor, the major subjects which are usually of high concern for the students, conduct sessions for repeated revisions to master all the primary skills that are of high importance while stepping</s>
Chunk: 2, Len: 100
<s>Do you offer year 7 classes?</s></s> the Year 7 high school. Our Year 7 English Tutor and Year 7 Maths tutor, the major subjects which are usually of high concern for the students, conduct sessions for repeated revisions to master all the primary skills that are of high importance while stepping ahead in the high school. We introduce the Year 7 topics to the students slowly and steadily so that the students do not lag behind nor become sha

# Mapping tokens back to original character posiitons

Input mapping allows model to map token predictions back to character spans in the original sequence

In [13]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True, # makes each chunk of context own feature, lets us map feature to exact sample it originated from
    return_offsets_mapping=True,    # returns tuple of (start_char, end_char) for each tuple, lets us trace tokens back to the original text
)
print(inputs.keys())
print(f"The 1 example gave {len(inputs['input_ids'])} features (feature = each tokenized input, chunk) that will be fed to model")
print(f"Type of inputs[0]: {type(inputs[0])}")
print("Input mappings: ", inputs['offset_mapping'][0]) # Showing chars of first chunk - restarts at 9 because now context (truncated)

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])
The 1 example gave 4 features (feature = each tokenized input, chunk) that will be fed to model
Type of inputs[0]: <class 'tokenizers.Encoding'>
Input mappings:  [(0, 0), (0, 2), (3, 6), (7, 12), (13, 17), (18, 19), (20, 27), (27, 28), (0, 0), (0, 0), (0, 3), (4, 9), (10, 14), (15, 19), (20, 21), (22, 29), (30, 33), (34, 38), (38, 39), (40, 48), (49, 52), (53, 61), (62, 64), (65, 71), (72, 73), (74, 80), (81, 84), (85, 96), (97, 107), (108, 111), (112, 115), (116, 124), (124, 125), (126, 128), (129, 131), (132, 135), (136, 139), (140, 148), (149, 152), (153, 156), (157, 166), (167, 172), (173, 177), (178, 179), (180, 187), (188, 194), (195, 198), (199, 207), (208, 212), (213, 216), (217, 221), (222, 223), (224, 228), (229, 235), (235, 236), (237, 240), (241, 245), (246, 247), (248, 255), (256, 259), (259, 261), (262, 265), (266, 270), (271, 272), (273, 277), (277, 278), (279, 284), (284, 285), (2

Example showing how it points to each chunk to know where it came from

In [14]:
inputs = tokenizer(
    dataset["train"][0:4]["question"],
    dataset["train"][0:4]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features (feature = chunk to be fed to model).")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")
print(type(inputs))

The 4 examples gave 8 features (feature = chunk to be fed to model).
Here is where each comes from: [0, 0, 0, 0, 1, 2, 2, 3].
<class 'transformers.tokenization_utils_base.BatchEncoding'>


As we can see, the first three examples (at indices 2, 3, and 4 in the training set) each gave four features and the last example (at index 5 in the training set) gave 7 features.

This information will be useful to map each feature we get to its corresponding label. As mentioned earlier, those labels are:

- (0, 0) if the answer is not in the corresponding span of the context
- (start_position, end_position) if the answer is in the corresponding span of the context, with start_position being the index of the token (in the input IDs) at the start of the answer and end_position being the index of the token (in the input IDs) where the answer ends

Steps
1. To determine which of these is the case and, if relevant, the positions of the tokens, we first find the indices that start and end the context in the input IDs (sequence_ids() method of the BatchEncoding our tokenizer returns)
2. Once we have those token indices, we look at the corresponding offsets, which are tuples of two integers representing the span of characters inside the original context.

-> We can thus detect if the chunk of the context in this feature starts after the answer or ends before the answer begins (in which case the label is (0, 0)). If that’s not the case, we loop to find the first and last token of the answer:

# Aligning character answer spans with token positions

In [15]:
start_positions = []  # initalizing list for storing start
end_positions = []  # initalizing list for storing end

""" the added part if breaks """
answers = dataset["train"]["answers"]

# for chunkNumber, offset_mapping (char mapping of each word to original context)
for i, offset in enumerate(inputs["offset_mapping"]): # loop thru each chunks offstep mapping
    if i == 0:
        print("-- Example of i=0 --")
        print(f"offset:{offset}\n") # showing offset

    # Mapping feature to exact sample it came from
    sample_idx = inputs["overflow_to_sample_mapping"][i]    # original dataset sample that chunk is

    if i == 0: print(f"-- Mapping feature to sample it came from --\n{inputs['overflow_to_sample_mapping'][i]}\n")

    answer = answers[sample_idx]  # assigns pecific answer for the current chunk (feature)
    if i == 0: print(f"-- Features and their Answers --\nFeature: {sample_idx}, Answer: {answer}\n")

    # get start_char and end_char
    start_char = answer["answer_start"][0]  # 0 bcos we set new answer each time
    end_char = answer["answer_start"][0] + len(answer["text"][0])

    # for each token in chunk, which part of input it came from? 0 = Q, 1 = Context, None = Special Token (SEP, CLS)
    sequence_ids = inputs.sequence_ids(i)
    if i == 0: print(f"--Sequence ID of 0 chunk--\n{sequence_ids}\n")

    # Find start and end of context within chunk
    idx = 0 # index
    while sequence_ids[idx] != 1:   # increment while None (special token) or Question (0)
        idx += 1
    context_start = idx # becomes one, set start
    while sequence_ids[idx] == 1: # hit context
        idx += 1
    context_end = idx - 1   # become None, set end to -1
    if i == 0: print(f"-- Context start and end of i=0 --\ncontext_start: {context_start}, context_end: {context_end}\n")

    # if the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # otherwise it's the start and end token positions
        idx = context_start # resetting idx (was context end from above)
        # UNTIL idx (set as context_start) increments to ans start
        while idx <= context_end and offset[idx][0] <= start_char:  # while index <= context_end and offset_mapping[index][startNo.] (here, 13) <= ans start
            idx += 1
        start_positions.append(idx - 1) # -1 bcos looping to past number

        idx = context_end
        # UNTIL idx (set as context_end) decrements to ans end
        while idx >= context_start and offset[idx][1] >= end_char:  # while idx >= context_start and offset_mapping[index][endNo.] >= ans end
            idx -= 1
        end_positions.append(idx + 1)   # +1 bcos looping to before number

print('-- idx of tokens within chunk -- \nstart end top bottom. If 0 then answer not fully inside context')
print(f"{start_positions}\n{end_positions}")

-- Example of i=0 --
offset:[(0, 0), (0, 2), (3, 6), (7, 12), (13, 17), (18, 19), (20, 27), (27, 28), (0, 0), (0, 0), (0, 3), (4, 9), (10, 14), (15, 19), (20, 21), (22, 29), (30, 33), (34, 38), (38, 39), (40, 48), (49, 52), (53, 61), (62, 64), (65, 71), (72, 73), (74, 80), (81, 84), (85, 96), (97, 107), (108, 111), (112, 115), (116, 124), (124, 125), (126, 128), (129, 131), (132, 135), (136, 139), (140, 148), (149, 152), (153, 156), (157, 166), (167, 172), (173, 177), (178, 179), (180, 187), (188, 194), (195, 198), (199, 207), (208, 212), (213, 216), (217, 221), (222, 223), (224, 228), (229, 235), (235, 236), (237, 240), (241, 245), (246, 247), (248, 255), (256, 259), (259, 261), (262, 265), (266, 270), (271, 272), (273, 277), (277, 278), (279, 284), (284, 285), (286, 289), (290, 295), (296, 304), (305, 310), (311, 314), (315, 322), (323, 325), (326, 330), (331, 338), (339, 342), (343, 346), (347, 355), (355, 356), (357, 364), (365, 373), (374, 377), (378, 386), (387, 396), (397, 399),

# Preprocessing Training Examples

In [16]:
max_length = 384    # NEW -- max length of chunk
stride = 128    # NEW -- stride (sliding window) size


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]   # NEW -- removes whitespaces (leading and trailing)
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True, # which feature came from which chunk
        return_offsets_mapping=True,    # character spans
        padding="max_length",   # NEW -- pads to max_length with (0's or PAD token) if shorter, or truncates
    )

    # print(type(inputs)) # <class 'transformers.tokenization_utils_base.BatchEncoding'>
    offset_mapping = inputs.pop("offset_mapping")   # character spans
    sample_map = inputs.pop("overflow_to_sample_mapping")   # which feature came from which chunk
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]  # feature it came from
        answer = answers[sample_idx]    # ans
        start_char = answer["answer_start"][0]  # ans_start
        end_char = answer["answer_start"][0] + len(answer["text"][0])   # ans_end
        sequence_ids = inputs.sequence_ids(i)   # 0, 1, None of sequence (to find start of context)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:   # until seq id = 1
            idx += 1
        context_start = idx # set start
        while sequence_ids[idx] == 1:   # while seq id == 1
            idx += 1
        context_end = idx - 1   # set end once no longer 1

        # if the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # otherwise it's the start and end token positions
            idx = context_start
            # UNTIL idx (set as context_start) increments to ans start
            while idx <= context_end and offset[idx][0] <= start_char:  # while index <= context_end and offset_mapping[index][startNo.] (here, 13) <= ans start
                idx += 1
            start_positions.append(idx - 1) # -1 bcos looping to past number

            idx = context_end
            # UNTIL idx (set as context_end) decrements to ans end
            while idx >= context_start and offset[idx][1] >= end_char:  # while idx >= context_start and offset_mapping[index][endNo.] >= ans end
                idx -= 1
            end_positions.append(idx + 1) # +1 bcos looping to before number

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Preparing dataset for training

In [17]:
train_dataset = dataset["train"].map(  # applying fn over whole dataset to every sample (or batch of samples) in the dataset
    preprocess_training_examples,   # Giving map fn - tokenizes, creates input_ids, attn_mask, token type ids, start and end positions
    batched=True,   # Speed up - Pass a batch of examples to the function at once (as dictionaries of lists), instead of one-by-one
    remove_columns=dataset["train"].column_names,  # Removing no longer req original fields (id, question, context, answers)
)
print(len(dataset["train"]), len(train_dataset))  # checking count counts of each
print(dataset['train'][0])

Map: 100%|██████████| 39/39 [00:00<00:00, 441.78 examples/s]

39 39
{'id': '18', 'title': 'class type', 'context': 'Our entry into Year 7 English and Maths programs are designed to ensure a smooth and comfortable transition for the students. It is for the students who are finishing their Year 6 primary school and stepping into the Year 7 high school. Our Year 7 English Tutor and Year 7 Maths tutor, the major subjects which are usually of high concern for the students, conduct sessions for repeated revisions to master all the primary skills that are of high importance while stepping ahead in the high school. We introduce the Year 7 topics to the students slowly and steadily so that the students do not lag behind nor become shaky about it. As the students get used to the concepts and skills, the Year 7 English Tutor and Year 7 Maths tutor start nurturing and working on it further. Our tutors help the Year 6 students get accustomed to the techniques and methods used in the high school and help them learn to work on it', 'question': 'Do you offer yea

# Preprocessing Validation Data

In [18]:
def preprocess_validation_examples(examples):
    # examples is the dataset being fed in through map
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )


    sample_map = inputs.pop("overflow_to_sample_mapping")   # which feature came from which chunk
    example_ids = []

    for i in range(len(inputs["input_ids"])):   # for chunk in inputs
        sample_idx = sample_map[i]  # sample index = correct feature where chunk came from (0... etc)
        example_ids.append(examples["id"][sample_idx])  # assigns sample_idx to actual hash of the id in dataset (56be4db0acb8001400a502ec... etc)
        sequence_ids = inputs.sequence_ids(i)   # sequence_ids... 0, 1, None, i not sample_idx bcos sample can be from different chunk

        # Removing question and special tokens from chunks (leaving just context)
        offset = inputs["offset_mapping"][i]    # list of tuples for chunk i, (start_char, end_char) - char span for token in original text
        inputs["offset_mapping"][i] = [
        # for each token in chunk, if seq_id = 1 (context) keep it, otherwise set to None (if 0 or None (Q, or Token))
        # for index, actual in enumerate(offset)
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [19]:
validation_dataset = dataset["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["test"].column_names,
)
print(len(dataset["test"]), len(validation_dataset))
print(dataset['test'][0])

Map: 100%|██████████| 10/10 [00:00<00:00, 2108.22 examples/s]

10 10
{'id': '24', 'title': 'class type', 'context': 'Master Coaching is offering an exclusive Holiday Enhancement Course – designed for students who wish to hone their knowledge, and stay ahead of their classmates. The course is open to students from year 2 to year 12. It is the best way to enjoy your holidays, make the most of the festivities, as well as, upgrade and strengthen your school’s assignments and syllabus. Master Coaching’s Holiday Enhancement Course includes English and Math tutoring, along with courses to help in OC preparation and basic curriculum. Achieving academic success is the number one priority for both parents and students. Getting ahead in class and acquiring the highest ranking, is what every student aims for. Having said this, you can accomplish such a goal, if you get trained and assisted by the best. We at Master Coaching understand the value of student goals and their aspirations – and strive to guide our pupils towards the road of academic success. Our Ho

# Training

In [20]:
import collections  # provides powerful container data types (defaultdict)
# defaultdict: if you access a key that doesn’t exist yet, it will automatically create an empty list for you

import evaluate # hf evaluate library to measure performance

metric = evaluate.load("squad") # used by original model

# Evaluation metrics fn - Turning logits from model output into answer spans

In [21]:
import numpy as np
from tqdm.auto import tqdm  # bars for training to look nice :)
n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):  # takes model outputs (logits)
    example_to_features = collections.defaultdict(list) # using defaultdict bcos if we access a key that doesn’t exist yet, it will automatically create an empty list

    for idx, feature in enumerate(features):    # hash, chunk - which feature belongs to which context (sample)
        example_to_features[feature["example_id"]].append(idx)  # '56be4db0acb8001400a502ec': [0], '56be4db0acb8001400a502ed': [1]

    predicted_answers = []  # empty list
    for example in tqdm(examples):  # loop through original QA (not chunks) - want 1pred/Q (not per chunk)
        example_id = example["id"]
        context = example["context"]
        answers = []

        # loop through all features associated with that example
        for feature_index in example_to_features[example_id]:   # FOR ALL CHUNKS OF QUESTION
            start_logit = start_logits[feature_index] # get models answers
            end_logit = end_logits[feature_index] # get models answers
            offsets = features[feature_index]["offset_mapping"] # get offsets

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist() # sorting top 20 to be judged
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist() # sorting top 20

            for start_index in start_indexes: # try all valid combos
                for end_index in end_indexes:
                    # skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    # VALID - puts answer and logit score into answers list
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)  # answer list to be judged

        # select best answer
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples] # true label creation
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)  # F1 and exact match from HF lib

# Fine-tuning model

Redoing to have all in one place

In [22]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint_deepset = "deepset/roberta-base-squad2"
tokenizer_deepset = AutoTokenizer.from_pretrained(model_checkpoint_deepset)
model_deepset = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint_deepset)
tokenizer_deepset.is_fast   # double checking model has fast tokenizer implemented

model_checkpoint_bert = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer_bert = AutoTokenizer.from_pretrained(model_checkpoint_bert)
model_bert = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint_bert)
tokenizer_bert.is_fast   # double checking model has fast tokenizer implemented

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


True

In [23]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.get_device_name(0))  # Should print your GPU name

True
NVIDIA GeForce RTX 5070


In [24]:
from transformers import TrainingArguments  # holds all the training hyperparameters and behavior settings

args = TrainingArguments(
    eval_strategy="epoch",   # eval per...
    save_strategy="no",  # do NOT save - was taking up too much google colab storage space
    learning_rate=2e-5, # learning rate
    num_train_epochs=3, # no. epoch. 3 seemed to generalise best after trying 1-15. Higher kept getting worse
    weight_decay=0.01,  # regularisation: penalizes large weights, helps overfitting - standard small
    fp16=True,  # mixed-precision training: HF optimisations to speed up training. Req. GPU
    report_to='none', # cancel wand DB implementation, wasn't useful for this project
)

In [25]:
# DEEPSET
from transformers import Trainer

trainer = Trainer(
    model=model_deepset,    # model
    args=args,  # args from above
    train_dataset=train_dataset,    # trainin dataset
    eval_dataset=validation_dataset,    # eval dataset
    tokenizer=tokenizer_deepset,    # tokenizer
)
trainer.train()

C:\Users\petea\AppData\Local\Temp\ipykernel_20280\1545219770.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log


TrainOutput(global_step=15, training_loss=2.662186686197917, metrics={'train_runtime': 1.3294, 'train_samples_per_second': 88.007, 'train_steps_per_second': 11.283, 'total_flos': 22928790403584.0, 'train_loss': 2.662186686197917, 'epoch': 3.0})

In [26]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, dataset["test"])

100%|██████████| 10/10 [00:00<00:00, 1364.09it/s]


{'exact_match': 60.0, 'f1': 75.26801142965427}

# Context Matching

In [27]:
all_contexts = dataset['train']['context'] + dataset['test']['context'] # combining all contexts from the training and test sets
original_contexts = " ".join(all_contexts)  # combining

unique_contexts = list(set(all_contexts))   # removing duplicate contexts (do you vs teach vs offer creates a lot)
unique_contexts.sort()  
default_context = " ".join(unique_contexts) # combining new set
print(f"all_contexts length: {len(original_contexts)}, default_contexts length: {len(default_context)}")

all_contexts length: 25343, default_contexts length: 9716


In [28]:
possible_contexts = [
    # 0 - OC Classes
    "At Master Coaching Hurstville, we offer extensive Opportunity Class (OC) preparation courses that build fundamental core skills needed in partaking in the exam. The OC exam is broken down into 3 parts; Thinking skills, mathematical reasoning and reading comprehension. In our OC preparation program, students will learn and develop a strong conceptual understanding of all key concepts in Maths, English and Thinking skills. Students will receive one on one support where our skilful tutor will cater to each students weakness and work along with them to perform their best. Each week we provide mock exams for students to practice the test under exam conditions and will then receive detailed feedback on where to improve. Tutors will also go through each question in a detailed manner and ensures to keep up with the student’s pace, ensuring they do not fall behind and motivate them to excel and build confidence.",
    # 1 - one on one tutoring
    "At master coaching hurstville, we offer One-to-One tutoring which allows us to customise the learning experience to each student’s unique needs, provide tailored support and boost confidence",
    # 2 - science + 7, 8, 9
    "There is a wide range of skills and knowledge covered in year 7-10 science. It is no wonder that many children find it challenging. Having a full comprehension of what is being taught in these junior years will ensure your child is ready to take on the challenges of the HSC. We offer science classes for all years 7-10.",
    # 3 - y10 sci
    "There is a wide range of skills and knowledge covered in year 7-10 science. It is no wonder that many children find it challenging. Having a full comprehension of what is being taught in these junior years will ensure your child is ready to take on the challenges of the HSC. We offer science classes for year 10, as well as HSC Subjects such as Chemistry, Biology and Physics.",
    # 4 - holiday
    "Master Coaching is offering an exclusive Holiday Enhancement Course – designed for students who wish to hone their knowledge, and stay ahead of their classmates. The course is open to students from year 2 to year 12. It is the best way to enjoy your holidays, make the most of the festivities, as well as, upgrade and strengthen your school’s assignments and syllabus. Master Coaching’s Holiday Enhancement Course includes English and Math tutoring, along with courses to help in OC preparation and basic curriculum. Achieving academic success is the number one priority for both parents and students. Getting ahead in class and acquiring the highest ranking, is what every student aims for. Having said this, you can accomplish such a goal, if you get trained and assisted by the best. We at Master Coaching understand the value of student goals and their aspirations – and strive to guide our pupils towards the road of academic success. Our Holiday Enhancement Course is developed around this student intent."
    ]


In [40]:
from transformers import pipeline

# Load the pipeline
qa_pipeline = pipeline(
    "question-answering",
    model=model_deepset,
    tokenizer=tokenizer_deepset,
)

print("---")
user_question = "Do you offer private tutoring?"

print(f"Question: {user_question}")
valid_question=1

# oc
if "oc" in user_question.lower():                       # oc
  context = possible_contexts[0]
  print("oc - possible_contexts[0]")
# one or personal
elif "one" in user_question.lower() or "personal" in user_question.lower() or "private" in user_question.lower():                 # one or personal
  context = possible_contexts[1]
  print("one or personal - possible_contexts[1]")
# science
elif "science" in user_question.lower():                                                                # science
  if "7" in user_question.lower() or "8" in user_question.lower() or "9" in user_question.lower():          # 7, 8, 9
    context = possible_contexts[2]
    print("science + 7, 8, 9 - possible_contexts[2]")
  elif "10" in user_question.lower():                                                                       # 10
    context = possible_contexts[3]
    print("science + 10 - possible_contexts[3]")
  else:
    context = default_context                                                                               # default
    print("science else - default context")
# holiday
elif "holiday" in user_question.lower():                                                                  # holiday
  context = possible_contexts[4]
  print("holiday - possible_contexts[4]")
# else
else:
  valid_question=0

if valid_question==1:
  # mock input
  question = user_question
  # ask
  result = qa_pipeline({
      "context": context,
      "question": question
  })
else:
  result = "I'm here to help with Master Coaching Hurstville related questions only sorry! :("

print(f"Context: {context}")
print(result)
print(f'Chatbot Response: {result['answer'][0].upper()+result['answer'][1:].lower()}')

Device set to use cuda:0


---
Question: Do you offer private tutoring?
one or personal - possible_contexts[1]
Context: At master coaching hurstville, we offer One-to-One tutoring which allows us to customise the learning experience to each student’s unique needs, provide tailored support and boost confidence
{'score': 0.2856055796146393, 'start': 31, 'end': 59, 'answer': 'we offer One-to-One tutoring'}
Chatbot Response: We offer one-to-one tutoring
